In [1]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time,threading
import pandas as pd
import numpy as np
from selenium.webdriver import ChromeOptions
#from selenium.webdriver.support import expected_conditions as EC
#adding custom headers so that FB can't detect this as a bot
opts=ChromeOptions()
opts.add_argument("--incognito")
opts.add_argument("--Host= https://www.google.com/")
opts.add_argument("--Connection= keep-alive")
opts.add_argument("--Accept= text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8")
opts.add_argument("--Accept-Encoding= gzip, deflate, sdch")
opts.add_argument("--Accept-Language= en-US,en;q=0.8")
opts.add_argument("--Referrer= https://www.google.com/")
opts.add_argument("--disable-infobars")
#browser= webdriver.Firefox(executable_path='C:/Users/r.m.karim/Desktop/Webdrivers/geckodriver')

#how_many_clicks=1500
#ee_more_posts=browser.find_element_by_xpath('//*[@id="m_more_item"]/a')
# to try to scrape for unlimited time....................................................
flag=1
file=pd.read_csv("prox.txt",sep='\t')
#random UA generator
def get_random_ua():
    random_ua = ''
    ua_file = 'ua_file.txt'
    try:
        with open(ua_file) as f:
            lines = f.readlines()
        if len(lines) > 0:
            prng = np.random.RandomState()
            index = prng.permutation(len(lines) - 1)
            idx = np.asarray(index, dtype=np.integer)[0]
            random_ua = lines[int(idx)]
    except Exception as ex:
        print('Exception in random_ua')
        print(str(ex))
    finally:
        return random_ua

opts.add_argument("--user-agent={}".format(get_random_ua()))
#opts.add_argument("--headless")
#random_ip=np.random.choice(file["IP Address"])
#opts.add_argument("--proxy-SERVER={0}:{1}".format(random_ip,file.Port.loc[file[file["IP Address"]==random_ip].index[0]]))
browser = webdriver.Chrome('C:/Users/r.m.karim/Desktop/Webdrivers/chromedriver',options=opts)
browser.get("https://m.facebook.com/groups/dsdbangladesh/")
browser.set_page_load_timeout(300)
def browsing():
    global flag
    while flag==1:
        opts.add_argument("--user-agent={}".format(get_random_ua()))
#       random_ip=np.random.choice(file["IP Address"])
#       opts.add_argument("--proxY-SERVER={0}:{1}".format(random_ip,file.Port.loc[file[file["IP Address"]==random_ip].index[0]]))
        #browser = webdriver.Chrome('C:/Users/r.m.karim/Desktop/Webdrivers/chromedriver',options=opts)
        #print(browser.execute_script("return navigator.userAgent;"))
        browser.implicitly_wait(120)
        browser.find_element_by_xpath('//*[@id="m_more_item"]/a').click() #chrome
        #browser.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[4]/div[2]/a').click()  #firefox
        time.sleep(abs(np.random.normal(5))+1)
        if flag==False:
            source_data=browser.page_source
            soup=bs(source_data,'lxml')
            paragraph = ["".join(x.findAll(text=True)) for x in soup.findAll("p")]
            df=pd.DataFrame({'Comments':paragraph})
            df.to_excel("DSD_Posts0.xls",index=False)
            print("Yes!Posts are scrapped!")
            print("restarting!sleeping for 20 seconds...")
            time.sleep(20)
            clicks=100
            while clicks <=100:
                browser.find_element_by_xpath('//*[@id="m_more_item"]/a').click()
                time.sleep(abs(np.random.normal(5))+1)
                clicks=+1
            source_data=browser.page_source
            soup=bs(source_data,'lxml')
            paragraph = ["".join(x.findAll(text=True)) for x in soup.findAll("p")]
            df=pd.DataFrame({'Comments':paragraph})
            df.to_excel("DSD_Posts1.xls",index=False)
            print("Yes!Posts are scrapped again!!")
            
            
def user_input():
    global flag
    keystrk=input('Please press ENTER to stop \n')
    # thread doesn't continue until key is pressed
    print('Crawling is stopped!')
    flag=False
    print("Now processing the scraped data so far! please wait...")
n=threading.Thread(target=browsing)
i=threading.Thread(target=user_input)
n.start()
i.start()